In [3]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'02.24.2022'

In [4]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [5]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [6]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [7]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df.head()

df.to_excel("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

# Create the Constraint Problem

Goal: Maximize DK Points

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [8]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [9]:
# number of players constraint
prob += pulp.lpSum(player_var for player_var in player_vars) == 8
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    

prob += get_position_sum(player_vars, df, 'PG') >= 1
prob += get_position_sum(player_vars, df, 'SG') >= 3
prob += get_position_sum(player_vars, df, 'G') >= 1
prob += get_position_sum(player_vars, df, 'SF') >= 1
prob += get_position_sum(player_vars, df, 'PF') >= 1
prob += get_position_sum(player_vars, df, 'F') >= 3
# prob += get_position_sum(player_vars, df, 'C') <= 2
prob += get_position_sum(player_vars, df, 'C') >= 1

# total salary constraint
prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

# finally, specify the goal
prob += pulp.lpSum([df['DK Ceiling'].iloc[i] * player_vars[i] for i in range(len(df))])

In [10]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [11]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
result = []
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Ceiling'], row['Name + ID'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Ceiling']
        result.append([row.Player, row['Roster Position'], row['Name + ID']])
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['SG', 'G', 'UTIL'] Devin Booker PHX 8800.0 58.6 Devin Booker (21388619)
['C', 'UTIL'] Nikola Jokic DEN 12500.0 72.2 Nikola Jokic (21388574)
['SF', 'PF', 'F', 'UTIL'] Josh Hart POR 7200.0 48.4 Josh Hart (21388662)
['PG', 'G', 'UTIL'] Rajon Rondo CLE 3900.0 34.3 Rajon Rondo (21388953)
['SF', 'F', 'UTIL'] De'Andre Hunter ATL 4500.0 34.4 De'Andre Hunter (21388870)
['PF', 'F', 'UTIL'] James Johnson BKN 3500.0 30.9 James Johnson (21389012)
['SG', 'SF', 'F', 'G', 'UTIL'] Seth Curry BKN 6000.0 40.0 Seth Curry (21388742)
['PG', 'SG', 'G', 'UTIL'] Gary Payton II GSW 3400.0 30.1 Gary Payton II (21389049)


(49800.0, 348.90000000000003)

In [109]:
print(result)

[['Lance Stephenson', ['PG', 'SF', 'F', 'G', 'UTIL'], 'Lance Stephenson (21347484)'], ['Clint Capela', ['C', 'UTIL'], 'Clint Capela (21347341)'], ['Trae Young', ['PG', 'G', 'UTIL'], 'Trae Young (21347148)'], ["D'Angelo Russell", ['PG', 'SG', 'G', 'UTIL'], "D'Angelo Russell (21347226)"], ['Jonas Valanciunas', ['C', 'UTIL'], 'Jonas Valanciunas (21347212)'], ['Devin Booker', ['SG', 'G', 'UTIL'], 'Devin Booker (21347168)'], ['Maxi Kleber', ['PF', 'C', 'F', 'UTIL'], 'Maxi Kleber (21347555)'], ['Terance Mann', ['SG', 'SF', 'F', 'G', 'UTIL'], 'Terance Mann (21347402)']]


In [98]:
new_list = result
df = pd.DataFrame(new_list)
df.to_excel("test_result.xlsx")

In [99]:
os.getcwd()

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\PlayerPool Analysis'